# Storing Temporal Data
In general, the value of **time series** data is in its retrospective (batch ingestion model), rather than in the live streaming of data. For this reason, storing **time series** data is necessary for most analyses.

A good storage solution is one that allows for easy access and reliability of data without requiring a large investment of computing resources. Later, we will look at what aspects of a dataset we should consider for storage, as well as examine the advantages of SQL databases, NoSQL databases, and a variety of flat file formats.

Developing a general **time series** storage solution is challenging because there are many different types of data, each with different storage, read/write, and analysis patterns. Some data will be stored and examined repeatedly, while others are only useful for a short period of time, after which they may be deleted entirely.

Use case examples:

<ins>*Use case 1˚:*</ins>
- We are collecting performance metrics on a production system. These performance metrics need to be stored for years on end, but the older the data gets, the less detailed it needs to be. Therefore, a storage medium is needed that automatically performs *downsampling* and separates the data as the information becomes old;

<ins>*Use case 2˚:*</ins>
- We have remote access to an open source repository of **time series** data, but we need to keep a local copy on your computer to reduce network traffic. The remote repository stores each time series in a folder of downloadable files on a web server, but we would like to compile all of these files into a single database to simplify things. The data must be immutable and capable of being stored indefinitely, as the aim is to have a reliable copy of the remote repository;

<ins>*Use case 3˚:*</ins>
- We create our own **time series** data by integrating a variety of data sources at different time scales, and with distinct pre-processing and formatting. Data collection and processing were tiring and time-consuming. We would like to store the data in its final format instead of running a pre-processing step successively, but we would also like to keep the raw data, to later explore pre-processing alternatives. You may need to re-examine the processed and raw data frequently as you develop new machine learning models, refitting new models on the same data, and also adding data over time as newer raw data becomes available. No need to downsample or separate data in storage.

Use cases solutions:

<ins>*Importance of how performance scales with size*</ins>
- in the first use case, we would look for a solution that could incorporate automated scripts to delete old data. We wouldn't be concerned about how the system scales to large datasets, as we plan to keep the dataset small. For the second and third case, we would expect to have a large and stable collection of data or a large and growing collection of data, respectively;

<ins>*Importance of random access versus sequential access of data points*</ins>
- in the second case, we expect all data to be accessed in equal parts, since this **time series** data would all have the same "age" upon insertion and would all reference the relevant data set. In contrast, in the first and third cases, we expect the most recent data to be accessed more frequently;

<ins>*Importance of automation scripts*</ins>
- apparently, the first case can be automated, while the second case would not require automation (since the data would be immutable). The third case suggests little automation, and also a considerable amount of data collection and processing of all parts of the data, not just the most recent ones. In the first case, we want a storage solution that can be integrated with scripts or stored procedures, while in the third case we want a solution that allows easy customization of data processing;

## Defining the Requirements

When considering storing **temporary series** data, we invite you to ask a few questions:
- <ins>*How much **time series** data will we store?* *How quickly will this data grow?*</ins>
    - We will want to choose an included storage solution for the expected growth rate of the data. Database administrators who are migrating from transaction-oriented datasets to **time series** are not infrequently surprised by how quickly the datasets can grow;<br><br>

- <ins>*Do frequencies typically have unlimited channels of updates (e.g., a constant stream of web traffic updates) or different events (e.g., a series of air traffic schedules for every major U.S. holiday over the past ten years)?*</ins>
    - If the data is like an unlimited channel, we will see more recent data. On the other hand, our data is a collection of **time series** separated into separate events, so events further apart in time can still be quite interesting. In the latter case, random access is the most likely pattern;<br><br>
    
- <ins>*Does the data have regular or irregular spacing?*</ins>
    - If the data is regularly spaced, we will be able to calculate more accurately and in advance how much data we expect to collect and how frequently this data will be entered into the system. If the data is irregularly spaced, we will use a less predictable data access style, which can efficiently facilitate periods of inactivity and periods of writing activity;<br><br>
    
- <ins>*Will we collect data continuously or will we have a well-defined end date?*</ins>
    - If we have a well-defined end date for data collection, it will be easier to know the size of the data set that needs to be accommodated. But after starting to collect a specific type of **time series**, several organizations discover that they no longer want to stop;<br><br>
    
- <ins>*What will we do with our **time series**? Is real-time views necessary? Preprocessed data for a neural network to iterate thousands of times? Fragmented data highly available to a large mobile user base?*</ins>
    - Whether the primary use case will indicate whether you are more likely to need sequential or random access to your data and the importance of a latency factor for choosing the storage format;<br><br>
    
- <ins>*How will we separate or downsample the data? How will we avoid infinite growth? What should be the life cycle of an individual data point in a **time series**?*</ins>
    - It is impossible to store all events forever. It is better to make decisions about systematic data deletion policies in advance than to do so in a one-off fashion. The more you anticipate, the better the choice you can make regarding storage formats. In the next section, we will talk more about this.<br><br>
    

The answers to these questions will indicate whether you should store raw or processed data, whether data should be entered into memory according to time or some other axis, and whether you need to store your data in a format that makes it easy to read and write. them. Use cases vary, so we must create a new inventory for each new set of data.

## Live Data versus. Stored Data

When thinking about which storage options are right for your data, it's critical to understand its lifecycle. The more realistic you are about your use cases, the less data we will need to save and the less time we will spend finding the ideal storage system, as we will not be scaling across an intractable amount of data. Organizations often over-record events of interest as they fear losing their data stores. However, having more data stored in an intractable form is less useful than having aggregated data stored over meaningful timescales.

When it comes to short-lived live data, like performance data that will be examined just to make sure nothing is wrong, we may never need to store the data in the form in which it is collected, at least not for long. This is more suited to event-driven data, where no single event is important and, instead, aggregated statistics are the values of interest.

Suppose we are running a web server that records and reports the amount of time it took each mobile device to fully load a given web page. The resulting irregularly spaced **time series** might look similar to the following table:

| Timestamp                       | Time to load the page |
| :------------------------------ | --------------------- |
| April 5, 2018 10:22:24 pm       | 23s                   |
| April 5, 2018 10:22:28 pm       | 15s                   |
| April 5, 2018 10:23:02 pm       | 14s                   |

<br>
<br>

Por diversas razões, talvez não estejamos interessados em nenhuma medição individual do tempo para carregr uma página. Gostaríamos de agregar os dados (digamos, tempo médio de carregar a página por minuto) e mesmo as estatísticas agregadas seriam interessantes apenas por um breve espaço de tempo. Para ter certeza de que podemos mostrar que o desempenho foi bom enquanto cuidava de tudo. Seria possível simplificar isso em um ponto de dados.

Instead of having 3,470 individual events that are of no interest to anyone, we will have compact and readily accessible values of interest. It is necessary to simplify data storage through aggregation and deduplication whenever possible.

| Período                      | Most accessed time | Loaded pages | Average time to load | Maximum charging time |
| :--------------------------- | ------------------ | ------------ | -------------------- | --------------------- |
| April 5, 2018 8pm - 8am      | 11pm               | 3.470        | 21s                  | 45s                   |

### Variables that change gradually

If you are storing a state variable, consider recording only the data points where the value changed. for example, if you are recording temperature in five-minute increments, your curve may look like a *step function*, especially if you only care about one value, such as the nearest degree. In this case, it is not necessary to store repetitive values, which ends up saving storage space.

### Noisy and high-frequency data

If the data is noisy, there are reasons not to care much about any specific data point. You may want to aggregate the data points before recording them, as the high noise level devalues ​​any individual measurements. Of course this will be determined by domain specificity and you will need to ensure that downstream users are still able to evaluate the noise in their measurements.

### Obsolete data

The older the data, the less likely your organization will use it in general. Whenever you start recording a new set of **time series** data, it is important to be aware of when it will become irrelevant:

- is there an expiry date?
- if it doesn't exist, can we investigate the analytics department's past research and see how far back in the past it realistically goes?

If we can automate data deletion in a way that doesn't invalidate data analysis actions, your storage options will be better and we will reduce the importance of scaling or slowing down queries on *bloat* datasets.

So far, we've looked at general use cases for **time series** storage. We also evaluate a set of queries related to how a **time series** data set will be generated and analyzed, so that these queries can inform our selection of a storage format. Now we'll look at two common options for storing **time series**:
- databases;
- files;

## Database Solutions

For almost every data analyst or engineer, a database is an intuitive and familiar solution when it comes to how to store data. As with relational data, a database is often a good storage option for **time series** data. This is especially true if you want an innovative solution to any of these classic database properties:
- a storage system that can be scaled across multiple servers;
- a low latency read/write system;
- functions already implemented to calculate commonly used metrics (such as calculating the average of a group-by query, where group-by can be applied to time metrics);
- troubleshooting and monitoring tools that can be used to improve system performance and analyze bottlenecks;

These are, among many others, good reasons to opt for a database over a file system, and we should think about a database solution for data storage, even more so when working with new data sets. A database, particularly a NoSQL database, can help ensure flexibility. What's more, a database will get your project up and running sooner than if you were working with individual files, because much of the necessary boilerplate code will already be running. Even if you ultimately choose a file storage solution, working with a database first can help you determine how to structure your own files as your new data processes become stable.

### SQL vs. NoQL

In the **time series** database community, the debate between SQL vs. NoSQL is intense and pervasive. Many database administration experts insist that SQL is the best standard database language and that there is no data, regardless of format, that cannot be well described by a good set of relational tables. Despite this, in practice, when organizations attempt to scale SQL solutions to accommodate large amounts of **time series** data, performance degradation is common. For this reason, it's also worth considering a NoSQL solution, especially if you're looking for a solution that can scale to accommodate cases where **time series** data collection begins with no finite time horizon in sight.

Although SQL and NoSQL solutions can be good for **time series** data, what motivates our debate are the difficulties of applying database logic to **time series** data, since this data differ from the type of data for which SQL databases were developed.

#### Data properties that originally influenced SQL databases

We can better understand the incompatibility between SQL and **time series** data by revisiting the history of SQL solutions. SQL solutions are based on transactional data, any necessary data that fully represents a discrete event. A transaction is made up of attributes that portray many primary keys, such as product, participants, time, and transaction value. Note that time can be present as a primary key, but only as one among many and not as a privileged axis of information. There are two important elements of transactional data that are quite different from **time series** needs:
- existing data points are updated frequently;
- data is accessed somewhat randomly, as there is no need for underlying ordering;

#### Properties of time series data

**Time series** data details the entire history of something, whereas a transaction record only tells us the final state. Therefore, **time series** data typically does not require updating, which means that random access for logging operations is of low priority.

Roughly speaking, the performance objectives fundamental to decades of SQl database design are not very important for **time series** databases. When considering the goals and designing these databases, we have quite a few priorities because of how we will use our **time series** data. The key features of our data use case are:
- writing operations take precedence over reading operations;
- data is not recorded, read or updated in random order, but in the order related to temporal sequencing;
- the probability of simultaneous readings is greater in **time series** data than in transactional data;
- there are few primary keys, at most, other than time itself;
- bulk deletions are much common than deletions of individual data points;

NoSQL databases support all of these features, as many of these general-purpose databases enable precisely what we want in **time series** databases, especially the emphasis on record operations over read operations. Conceptually, NoSQL databases map well to **time series** data because they natively portray aspects of data collection, such as not all fields being collected for all data points. Flexible NoSQL schemas work naturally with **time series** data. Much of the data that has led to the current popularity of NoSQL databases is **time series** data.

Therefore, NoSQL databases tend to have greater performance than SQL databases in write operations.

#### How to choose between SQL and NoSQL

We may be under the impression that NoSQL is the best choice, but there are many use cases for SQL databases. It is important to keep in mind that "data that is usually requested at the same time must be stored in the same location"

Ultimately, the differences between SQL and NoSQL are quite implementation dependent and are not as systematic or important as is often believed. Follow your data's guidance on the specific implementation of one of these technologies. When considering the attributes of your time series data and access usage patterns, keep in mind some general limitations:

*Advantages of SQL for **time series***:
- if a **time series** is stored in an SQL database, we will be able to easily relate it to relevant cross-sectional data also stored in that database;
- hierarchical **time series** data is a natural fit for relational tables. A suitable set of SQL schemas will help group related time series and clearly delimit the hierarchy, whereas they could be distributed less systematically in a NoSQL solution;
- if you are preparing a **time series** based on transactional data, where the data is best stored in an SQL database, it will be advantageous to also store the **time series** in the same database for ease checking and crossing data;

*Advantages of NoSQL for **time series***:
- writing is fast;
- if we do not know the future data in depth, NoSQL is more advantageous for designing an intelligent and robust schema;
- for the inexperienced user, these banks are usually an innovative and better-performing solution, as we are likely to design a clumsy scheme or get stuck in a project with a mediocre scheme;

### Popular Database and Time Series Archive Solutions

Now we will look at some database solutions for **time series** data. They will give us an idea of ​​what is available beyond traditional SQL solutions. Note that the technologies discussed are part of a competitive and fragmented technological scenario. What is commonly used this year may not be popular next. Therefore, this analysis should not be seen as a specific technology recommendation, but rather as a set of samples that help illustrate the current state of the market.

#### Specific time series database and related monitoring tools

First, we will cover tools developed specifically for storing and monitoring **time series** data. In particular, we'll take a look at a **time series** database (InfluxDB) and another product, a performance monitoring tool that can double as a **time series** storage solution (Prometheus). The advantages of each tool necessarily reflect its emphasis and different usage patterns.

##### InfluxDB

InfluxDB is a specific **time series** database, according to the description (https://oreil.ly/6qmVH) on the project page on GitHub:
- "InfluxDB is an open source **time series** database...., Useful for recording metrics, events and performing analysis."
    
In InfluxDB, data is organized by **time series**. A data point in InfluxDB consists of:
- a timestamp;
- a label indicating what the measurement consists of;
- one or more key/value fields;
- key/value pairs containing metadata tags;

InfluxDB, as a time-aware database, automatically records the timestamps of any data point that arrives without a timestamp.

Other advantages of InfluxDB:
- data retention options that make it possible to easily automate the classification and deletion of obsolete data;
- high speed of data ingestion and intense data compression;
- possibility of tagging individual **time series** to allow quick indexing that correspond to a specific criteria;
- easy registration with the *TICK* stack (https://oreil.ly/KLjGo), a platform for capturing, storing, monitoring and displaying **time series** data.

There are many other databases specific to **time series**, but InfluxDB is one of the most popular, we will soon come across it. The options offered by it are those normally provided by specialized databases for **time series**, as these options portray the most desired attributes for data storage.

As a database, InfluxDB is a *push* based system, i.e. when using it, we push data to the database for ingestion. Which is very different from Prometheus.

##### Prometheus

Prometheus describes itself (https://github.com/prometheus/prometheus) as a "time series database and monitoring system" that works over HTTP. This description signals its general emphasis: first, monitoring. Second, storage. The great advantage of Prometheus is that it is a *pull*-based system, that is, the logic of how data is extracted to create a **time series** and the frequency with which it is centralized can be easily adjusted and inspected.

Prometheus is a great resource to have on hand during emergencies because it is concise and self-contained. However, there is no guarantee that it will be perfectly updated or accurate, due to the *pull*-based architecture. Even though it is the ideal technology for temporary performance monitoring, although not at all elegant, it is not suitable for uses where data needs to be 100% accurate. Prometheus is expressed in a functional language called *PromQL*.

Prometheus also offers, via PromQL, an API for many common **time series** tasks, including even sophisticated functions like making a prediction *(predict_lin ear())* and calculating the rate of increase per unit time * (rate())*. Aggregation by time periods also has a simple interface. Prometheus tends to emphasize monitoring and analysis over maintenance, so compared to InfluxDB, there is less automated data curation functionality.

Prometheus is a useful **time series** storage solution, especially for live streaming applications and when data availability is essential. It has a steeper learning curve due to the custom scripting language and less database-like architecture and API, but it is still widely used.

#### General NoSQL Databases

Although databases specifically for **time series** offer many advantages, you can also consider the general use of NoSQL databases. These types of databases are oriented towards *documents* and not *tables* (relational) and generally do not have many explicit and exclusive functions for **time series**.

However, the flexible schemas of NoSQL databases are a great help for **time series** data, especially for new projects where the pace of data collection and the number of input channels may change over time. course of the useful life of a data set. For example, a **time series** might start as one data channel, but gradually grow to include more data types, all with timestamps. Later, it may be decided that some input channels are not very useful and may be discontinued. In this case, storing the data in a SQL table would be difficult for several reasons and would result in many NaNs where the data was not available. A NoSQL database would simply ignore missing channels when unavailable, instead of marking the data store with a bunch of NaNs.

A popular and high-performance NoSQL **time series** database is MongoDB. Mongo is very aware of its value as a **time series** database and has a strong push to develop IoT-friendly architecture and instructions. It provides high-level aggregation capabilities that can be applied to time aggregation operations and temporal related groupings, and also offers many automated processes for dividing time into relevant user-oriented tags, such as day of the week and month:
- dayOfWeek;
- dayOfMonth;
- hour;

Furthermore, Mongo has undertaken extensive development work to demonstrate how **time series** can be handled. A prepared set of documentation and institutional focus on **time series** data explicitly means that users can expect this database to continue to develop even more time-friendly capabilities.

However, more useful than these features is Mongo's flexibility in evolving the schema over time. This schema flexibility will save you a lot of headaches if you work with rapidly evolving **time series** data with constantly changing collection practices.

When you need query and schema flexibility in an app, a NoSQL database will be the best option, in addition to providing a reasonable balance between specific and general time flexibility.

Another advantage of a general NoSQL database, rather than a database specific to **time series** data, is that we can more easily integrate non-**time series** data in the same place, aiming to facilitate the crossing of data from related sets. Often, a general-purpose NoSQL database is just the right combination of performance considerations and SQL-like functionality, without the intelligence required to optimize a SQL database schema for **Time Series** functionality.

## File Solutions

A database is software that integrates scripting and data storage. It is basically a flat file in a wrapper within a special software whose responsibility is to ensure and facilitate the use of this file.

Sometimes it makes sense to strip away this outer layer and take full responsibility for data storage. While this is uncommon in business applications, it is frequently done in scientific research and in rare industrial uses (such as high-frequency algorithmic trading) where speed is critical. In these cases, it is the analyst who will design a much more complex data pipeline involving allocating storage space, opening files, reading files, closing files, protecting files, and so on, rather than simply writing some database queries. of data.

A flat file solution is a good option if any of these conditions apply:

- data format is stable, so that we can commit to a specification for a reasonably long period of time;
- data processing is I/O, so it makes sense to spend time developing to speed it up;
- we don't need random access, but you can read data sequentially;

### NumPy

If the data is strictly numeric, a widely used option for storing this data is a Python **NumPy** *array*. **NumPy** *arrays* can be easily saved in a variety of formats and there are many benchmarking publications comparing their relative performance. For example, *array_storage_benchmark* from **GitHub** (https://perma.cc/ZBS7-PR56) was developed to test the efficiency and speed of various **NumPy** file formats

A disadvantage of **NumPy** *arrays* is that they support a single data type, which means that we cannot store heterogeneous **time series** data automatically. Therefore, we should think about whether having just one data type can work for raw or processed data (although there are ways to get around this restriction). Other disadvantages of *arrays* **NumPy** is that it is unnatural to add labels to rows or columns, so there is no direct way to record the timestamp of each row of an array, for example. 

The advantage of using *arrays* **NumPy** is that there are many options for saving one, including compressed binary format which takes up less space and has faster I/O than you will see with a database solution. It is also as innovative as possible in terms of performance data structure for analytics and storage.

### Pandas

If you want easy data labeling or storage of heterogeneous **time series** data (or both), consider the less optimized but more flexible **Pandas** *DataFrame*. If we are a **time series** made up of many different data types, perhaps including event counts (inits), state measures (floats), and labels (strings), it might be a good solution. In these cases, we'll probably stick with **Pandas**'s *DataFrame* (also remember that the name "Pandas" actually comes from the "panel data" elision, so this is a natural format for many use cases ). **Pandas** *DataFrame* are widely used and there are several online resources to compare the various formats (https://perma.cc/BNJ5-EDGM) used in data storage.

### Standard equivalents in R

The native formats for storing **R** objects are *.Rds* and *.Rdata*. They are binary file formats. As such, they will necessarily be more efficient for I/O compression than text-based formats and are similar to **Pandas** *DataFrames*. Similarly, the *feather* format (https://perma.cc/4C3J-TBK8) can be used in **R** and **Python** to save DataFrames in a language-independent file format. For **R** users, native binary formats will perform best. The following table compares file format options:

| Format   | Size | Relative time to load |
| :------- | ---- | --------------------- |
| .RDS     | 1x   | 1x                    |
| feather  | 2x   | 1x                    |
| csv      | 3x   | 20x                   |